## Test Mel Spectogram

### Import library

In [20]:
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import cv2
import os
import utils
import csv

### Spectogram

In [30]:
def mp3_to_melspec(mp3_file, output_path, *, sr=22050, duration=30, n_fft=2048, n_mels=244, hop_length=2713):

    y, sr = librosa.load(mp3_file, sr=sr, duration=duration)

    # calculete mel-spectogram
    mel_spec = librosa.feature.melspectrogram(
    y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels
    )
    print(f'mel_spec: {mel_spec}')
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    print(f'mel_spec_db: {mel_spec_db}')
    print(f'({mel_spec_db} - {np.min(mel_spec_db)}) / ({np.max(mel_spec_db)}-{np.min(mel_spec_db)})')
    normalized = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db)-np.min(mel_spec_db))  #

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    librosa.display.specshow(normalized, sr=sr, hop_length=hop_length)
    plt.savefig(str(output_path), bbox_inches='tight', pad_inches=0)
    # plt.close()

In [31]:
tracks = pd.read_csv('project_data/tracks.csv', index_col=0)
track_ids = tracks.index
track_ids = [107535, 2]
AUDIO_DIR = 'dataset/fma_small'
SPECT_PATH = 'project_data/spectrograms'

In [32]:
try:
    # Create spectrograms
    cnt = 0

    for i in track_ids:
        cnt += 1
        if cnt % 100 == 0:
            print(f"Processing track {cnt}...")
        path = utils.get_audio_path(AUDIO_DIR, i)
        output_file = utils.get_spectogram_path(SPECT_PATH, i)
        mp3_to_melspec(path, output_file)
except Exception as e:
    print(f"Error processing file {i}: {e}")

mel_spec: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
mel_spec_db: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
([[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] - 0.0) / (0.0-0.0)
Error processing file 107535: index -1 is out of bounds for axis 0 with size 0


/var/folders/42/12p7rrgx3zj15zz7yc0tm30w0000gn/T/ipykernel_94647/2135162986.py:13: RuntimeWarning: invalid value encountered in divide
  normalized = (mel_spec_db - np.min(mel_spec_db)) / (np.max(mel_spec_db)-np.min(mel_spec_db))  #


In [9]:
df = pd.read_csv('project_data/tracks.csv')

df['filename'] = df['track_id'].apply(lambda x: f'{x:06d}.png')
new_data = df[['filename', 'genre']]
new_data.to_csv('project_data/spectrogram.csv', index=False)